# Abstractive:
generate new text that captures the most relevant information.


In [1]:
#!pip install transformers da

In [2]:
#!pip install transformers datasets evaluate rouge_score

In [3]:
#!pip install --upgrade pip

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
# so we are using the billsum data set here
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test")

Found cached dataset billsum (C:/Users/vishal567795/.cache/huggingface/datasets/billsum/default/3.0.0/75cf1719d38d6553aa0e0714c393c74579b083ae6e164b2543684e3e92e0c4cc)


In [6]:
#Split the dataset into a train and test set with the train_test_split method:
billsum = billsum.train_test_split(test_size=0.2)

In [7]:
billsum["train"][0]

{'text': 'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 12804.9 of the Vehicle Code is amended to read:\n12804.9.\n(a) (1) The examination shall include all of the following:\n(A) A test of the applicant’s knowledge and understanding of the provisions of this code governing the operation of vehicles upon the\nhighways.\nhighways, including, but not limited to,\nprovisions relating to\nsafe overtaking and passing, including, but not limited to, Section 21753.\n(B) A test of the applicant’s ability to read and understand simple English used in highway traffic and directional signs.\n(C) A test of the applicant’s understanding of traffic signs and signals, including the bikeway signs, markers, and traffic control devices established by the Department of Transportation.\n(D) An actual demonstration of the applicant’s ability to exercise ordinary and reasonable control in operating a motor vehicle by driving it under the supervision of an examining offi

There are two fields that you’ll want to use:

text: the text of the bill which’ll be the input to the model.
summary: a condensed version of text which’ll be the model target.

# Preprocessing

In [8]:
# The next step is to load a T5 tokenizer to process text and summary:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

The preprocessing function you want to create needs to:

1.Prefix the input with a prompt so T5 knows this is a summarization task. Some models capable of multiple NLP tasks require prompting for specific tasks.
2.Use the keyword text_target argument when tokenizing labels.
3.Truncate sequences to be no longer than the maximum length set by the max_length parameter.

In [9]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

To apply the preprocessing function over the entire dataset, use 🤗 Datasets map method. You can speed up the map function by setting batched=True to process multiple elements of the dataset at once:

In [10]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)


Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

Now create a batch of examples using DataCollatorForSeq2Seq. It’s more efficient to dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [11]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint, return_tensors="tf")

# Evaluate

Including a metric during training is often helpful for evaluating your model’s performance. You can quickly load a evaluation method with the 🤗 Evaluate library. For this task, load the ROUGE metric (see the 🤗 Evaluate quick tour to learn more about how to load and compute a metric):

In [12]:
import evaluate

rouge = evaluate.load("rouge")

Then create a function that passes your predictions and labels to compute to calculate the ROUGE metric:

In [13]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

Your compute_metrics function is ready to go now, and you’ll return to it when you setup your training.

# Train

You’re ready to start training your model now! Load T5 with AutoModelForSeq2SeqLM:

In [14]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [15]:
#pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [16]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_billsum["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_billsum["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
c:\users\vishal567795\appdata\local\programs\python\python39\lib\site-packages\datasets\arrow_dataset.py:388: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [17]:
import tensorflow as tf

model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [18]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_test_set)

In [19]:
from transformers.keras_callbacks import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="my_awesome_billsum_model",
    tokenizer=tokenizer,
)

C:\Users\vishal567795\Desktop\Text_Summarizer\my_awesome_billsum_model is already a clone of https://huggingface.co/vishal567795/my_awesome_billsum_model. Make sure you pull the latest changes with `repo.git_pull()`.


In [20]:
callbacks = [metric_callback, push_to_hub_callback]

In [21]:
#model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3, callbacks=callbacks)      

In [22]:
text = """“The Two Temples”
A man visits two temples, one in New York and one in England. In part 1, Temple First, a man walks three miles to a temple only to find he can’t get in. The church warden refuses him entry because he looks too poor. The man notices a side door, which he thinks leads up to the tower. He sneaks in, climbing the fifty stone steps to the upper floor. He’s able to look through a gap in a window at the service. In part 2, Temple Second, a stranger in London who owes his landlord money needs something to do until he can sneak back to his room. He ends up at a theatre. He wants to go in but doesn’t have enough for a ticket. While thinking about pawning his overcoat, a man suddenly approaches and gives him a free ticket."""

In [23]:
from transformers import pipeline

summarizer = pipeline("summarization", model="stevhliu/my_awesome_billsum_model")
summarizer(text)

c:\users\vishal567795\appdata\local\programs\python\python39\lib\site-packages\transformers\generation\utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


[{'summary_text': 'In part 1, a man walks three miles to a temple only to find he can’t get in . the church warden refuses him entry because he looks too poor. The man notices a side door, which he thinks leads up to the tower.'}]

In [24]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("stevhliu/my_awesome_billsum_model")
inputs = tokenizer(text, return_tensors="tf").input_ids

In [25]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained("stevhliu/my_awesome_billsum_model",from_pt=True)
outputs = model.generate(inputs, max_new_tokens=100, do_sample=False)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFT5ForConditionalGeneration: ['lm_head.weight', 'encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
- This IS expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [26]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'a man visits two temples, one in New York and one in England. In part 1, Temple Second, a man walks three miles to a temple only to find he can’t get in. The church warden refuses him entry because he looks too poor. The church warden refuses him entry because he looks too poor. The man notices a side door, which he thinks leads up to the tower. He sneaks in,'

In [27]:

#!pip freeze requirement.txt